# 3. Methodology

In [28]:
import bs4 as bs
import urllib.request

source = urllib.request.urlopen('https://bostonpads.com/2019-boston-apartment-rental-market-report/').read()
soup = bs.BeautifulSoup(source,'lxml')

In [2]:
table = soup.find_all('table')[6]
table_rows = table.find_all('tr')

In [3]:
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

[]
['Fenway', '3,129', '3,108', '257', '8.27%']
['Symphony', '1,921', '1,906', '122', '6.40%']
['Roxbury', '722', '722', '45', '6.23%']
['Mission Hill', '2,126', '2,105', '122', '5.80%']
['East Boston', '1,631', '1,599', '76', '4.75%']
['All Areas', '150,057', '70,053', '2,537', '3.62%']
['City Of Boston', '121,304', '41,304', '1,730', '4.19%']
['Outside Boston', '29,874', '28,749', '807', '2.81%']


In [4]:
import pandas as pd

dfs = pd.read_html('https://bostonpads.com/2019-boston-apartment-rental-market-report/',header=0)
for df in dfs:
    print(df)

   Total Apartments Non Luxury  Total Available Apartments Non Luxury  \
0                        70053                                   2537   

  Real Time Availability Rate  Total Vacant Apartments  \
0                       3.62%                     1338   

   Total Vacant Non-Luxury  
0                      506  
Empty DataFrame
Columns: [Total Galleries, Total Pictures, Total Videos]
Index: []
                    Unnamed: 0 July 2017 July 2019 % Change
0       Real-Time Vacancy-Rate     2.91%     3.69%   21.14%
1  Average 1 Bedroom Apartment    $1,938    $2,068    6.29%
2  Average 2 Bedroom Apartment    $2,439    $2,594    5.98%
   REAL TIME VACANCY RATE  = Total Apartments Currently Vacant
0  REAL TIME VACANCY RATE  =      Total Apartments in Database
   REAL TIME AVAILABILITY RATE  =  \
0  REAL TIME AVAILABILITY RATE  =   

  (Total Apartments Currently Vacant + Apartments Set to Become Available on a Later Day)  
0                       Total Apartments in Database          

In [5]:
import json # library to handle JSON files

#conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

url="https://cocl.us/Geospatial_data"
address = 'boston, MA'

geolocator = Nominatim(user_agent="boston")
#geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston are 42.3602534, -71.0582912.


In [6]:
CLIENT_ID = 'PVLO1XLOLFHLK50CEARO2QPNTY1U0AP4HNYPJIIJODXAGWSS' # your Foursquare ID
CLIENT_SECRET = 'KK42LKUP0OSHVW1GB5JEWUDIOE5T3RSARPDNFIJJ3OLLSHHG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 500
#print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: PVLO1XLOLFHLK50CEARO2QPNTY1U0AP4HNYPJIIJODXAGWSS
CLIENT_SECRET:KK42LKUP0OSHVW1GB5JEWUDIOE5T3RSARPDNFIJJ3OLLSHHG


'https://api.foursquare.com/v2/venues/search?client_id=PVLO1XLOLFHLK50CEARO2QPNTY1U0AP4HNYPJIIJODXAGWSS&client_secret=KK42LKUP0OSHVW1GB5JEWUDIOE5T3RSARPDNFIJJ3OLLSHHG&ll=42.3602534,-71.0582912&v=20180604&radius=500&limit=100'

In [7]:
results = requests.get(url).json()

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.shape
#dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


(100, 25)

In [10]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Boston City Hall,City Hall,1 City Hall Sq.,at Congress St.,42.360367,-71.058004,"[{'label': 'display', 'lat': 42.36036733073828...",26,02201,US,Boston,MA,United States,"[1 City Hall Sq. (at Congress St.), Boston, MA...",NaN,4a942695f964a5208a2020e3
1,City Hall Plaza,Plaza,1 City Hall Sq,at Cambridge St,42.359652,-71.059477,"[{'label': 'display', 'lat': 42.35965223045038...",118,02201-1001,US,Boston,MA,United States,"[1 City Hall Sq (at Cambridge St), Boston, MA ...",NaN,4a74b8c9f964a5205edf1fe3
2,City of Boston Credit Union,Credit Union,1 City Hall Sq,NaN,42.360329,-71.057991,"[{'label': 'display', 'lat': 42.36032867431640...",26,02111,US,Boston,MA,United States,"[1 City Hall Sq, Boston, MA 02111, United States]",NaN,4cb1e179db32f04d0fa3cb4d
3,Steaming Kettle,Monument / Landmark,NaN,NaN,42.359420,-71.059431,"[{'label': 'display', 'lat': 42.35941997765921...",131,02203,US,Boston,MA,United States,"[Boston, MA 02203, United States]",NaN,4b91259af964a52086a633e3
4,The Freedom Trail,Historic Site,The Freedom Trail,btwn Tremont St & Essex St,42.357314,-71.061038,"[{'label': 'display', 'lat': 42.35731415901699...",397,02111,US,Boston,MA,United States,[The Freedom Trail (btwn Tremont St & Essex St...,NaN,4b41657df964a5203dc625e3


In [11]:
import pandas as pd
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
boston_venues = getNearbyVenues(names=dataframe_filtered['name'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

boston_venues.head()

Boston City Hall
City Hall Plaza
City of Boston Credit Union
Steaming Kettle
The Freedom Trail
Food Trucks at City Hall
Union Oyster House
Across The Border Food Truck
Old State House
Boston Transportation Department
Union Square Donuts
Uncommon Feasts
Texas Thunder BBQ Company at The BBQ Beach Party
Recreo Coffee Roasterie
Best Of Boston
Trixie's Palace
Carmen Park
Boston Night Market
Triangle Coffee
Richard's Barber Shop
Boston City Hall Farmers' Market
Hub On Wheels
Robert Scibilia Square
City Hall Deli (8th Floor)
Teri-Yummy food truck at City Hall.
Pearle Vision
Sears Crescent & Sears Block
The Patios
Boston Seasons
Big Apple Circus
Revelry Food Truck
Jantzen and Associates, P.C.
M&N Great Taste
James Michael Curley Park
Bill Russell Statue
Dreamland Wax Museum
Real Staffing Solutions
Arnold "Red" Auerbach Statue
Policy Room
Boston Redevelopment Authority Office
SUBWAY
Boston Greenfest Aug 15-17
Thermopylae
Dunkin'
Two Center Plaza
One Washington Mall
McDermott Will & Emery LLP
FB

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Boston City Hall,42.360367,-71.058004,Faneuil Hall Marketplace,42.359978,-71.056410,Historic Site
1,Boston City Hall,42.360367,-71.058004,Boston Public Market,42.361950,-71.057466,Market
2,Boston City Hall,42.360367,-71.058004,Saus Restaurant,42.361076,-71.057054,Belgian Restaurant
3,Boston City Hall,42.360367,-71.058004,Red's Best,42.361960,-71.057587,Seafood Restaurant
4,Boston City Hall,42.360367,-71.058004,Tatte Bakery & Cafe,42.358451,-71.057981,Bakery


In [15]:
print('{} venues were returned by Foursquare.'.format(boston_venues.shape[0]))
boston_venues.head(20)

9671 venues were returned by Foursquare.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Boston City Hall,42.360367,-71.058004,Faneuil Hall Marketplace,42.359978,-71.056410,Historic Site
1,Boston City Hall,42.360367,-71.058004,Boston Public Market,42.361950,-71.057466,Market
2,Boston City Hall,42.360367,-71.058004,Saus Restaurant,42.361076,-71.057054,Belgian Restaurant
3,Boston City Hall,42.360367,-71.058004,Red's Best,42.361960,-71.057587,Seafood Restaurant
4,Boston City Hall,42.360367,-71.058004,Tatte Bakery & Cafe,42.358451,-71.057981,Bakery
5,Boston City Hall,42.360367,-71.058004,Big Apple Circus,42.360208,-71.059269,Circus
6,Boston City Hall,42.360367,-71.058004,North End Park,42.362488,-71.056477,Park
7,Boston City Hall,42.360367,-71.058004,George Howell Coffee,42.361832,-71.057445,Coffee Shop
8,Boston City Hall,42.360367,-71.058004,Quincy Market,42.360106,-71.054881,Historic Site
9,Boston City Hall,42.360367,-71.058004,Boston Massacre Monument,42.358955,-71.056971,Monument / Landmark


In [16]:
boston_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
33 Bowker St,100,100,100,100,100,100
Across The Border Food Truck,65,65,65,65,65,65
Aris Eat Barbecue,100,100,100,100,100,100
"Arnold ""Red"" Auerbach Statue",95,95,95,95,95,95
Best Of Boston,100,100,100,100,100,100
...,...,...,...,...,...,...
United Night of Worship,100,100,100,100,100,100
Uptown Cafe,81,81,81,81,81,81
Verizon Wireless B2B,99,99,99,99,99,99


In [17]:
print('There are {} uniques categories.'.format(len(boston_venues['Venue Category'].unique())))

There are 121 uniques categories.


In [18]:
# one hot encoding
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighborhood'] = boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
manhattan_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

,American Restaurant,Aquarium,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,...,Tea Room,Thai Restaurant,Theater,Tourist Information Center,Track Stadium,Trail,Tunnel,Vegetarian / Vegan Restaurant,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
boston_grouped = boston_onehot.groupby('Neighborhood').mean().reset_index()
boston_grouped.head()

,Neighborhood,American Restaurant,Aquarium,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,...,Tea Room,Thai Restaurant,Theater,Tourist Information Center,Track Stadium,Trail,Tunnel,Vegetarian / Vegan Restaurant,Wine Shop,Yoga Studio
0,33 Bowker St,0.000000,0.000000,0.010000,0.0,0.0,0.010000,0.050000,0.010000,0.0,...,0.010000,0.0,0.0,0.0,0.0,0.000000,0.0,0.010000,0.000000,0.01
1,Across The Border Food Truck,0.000000,0.015385,0.015385,0.0,0.0,0.000000,0.015385,0.030769,0.0,...,0.015385,0.0,0.0,0.0,0.0,0.015385,0.0,0.015385,0.015385,0.00
2,Aris Eat Barbecue,0.030000,0.000000,0.010000,0.0,0.0,0.010000,0.050000,0.020000,0.0,...,0.010000,0.0,0.0,0.0,0.0,0.000000,0.0,0.010000,0.010000,0.02
3,"Arnold ""Red"" Auerbach Statue",0.031579,0.000000,0.010526,0.0,0.0,0.010526,0.042105,0.010526,0.0,...,0.010526,0.0,0.0,0.0,0.0,0.000000,0.0,0.010526,0.010526,0.00
4,Best Of Boston,0.010000,0.000000,0.010000,0.0,0.0,0.010000,0.050000,0.010000,0.0,...,0.010000,0.0,0.0,0.0,0.0,0.000000,0.0,0.010000,0.010000,0.02


In [21]:
num_top_venues = 5

for hood in boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----33 Bowker St----
                venue  freq
0  Italian Restaurant  0.19
1       Historic Site  0.07
2         Coffee Shop  0.06
3              Bakery  0.05
4                 Pub  0.04


----Across The Border Food Truck----
             venue  freq
0      Coffee Shop  0.08
1            Hotel  0.08
2    Historic Site  0.08
3             Park  0.06
4  Harbor / Marina  0.06


----Aris Eat Barbecue----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.06
2       Historic Site  0.06
3              Bakery  0.05
4               Hotel  0.04


----Arnold "Red" Auerbach Statue----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.07
2       Historic Site  0.06
3                 Pub  0.05
4      Sandwich Place  0.04


----Best Of Boston----
                venue  freq
0  Italian Restaurant  0.17
1       Historic Site  0.06
2         Coffee Shop  0.06
3               Hotel  0.05
4              Bakery  0.05


----Big Apple Circus----

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
import numpy as np # library to handle data in a vectorized manner
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boston_grouped['Neighborhood']

for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,33 Bowker St,Italian Restaurant,Historic Site,Coffee Shop,Bakery,Seafood Restaurant
1,Across The Border Food Truck,Coffee Shop,Hotel,Historic Site,Harbor / Marina,Park
2,Aris Eat Barbecue,Italian Restaurant,Coffee Shop,Historic Site,Bakery,Hotel
3,"Arnold ""Red"" Auerbach Statue",Italian Restaurant,Coffee Shop,Historic Site,Pub,Sandwich Place
4,Best Of Boston,Italian Restaurant,Historic Site,Coffee Shop,Bakery,Hotel


In [24]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 2

boston_grouped_clustering = boston_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 0, 1, 1, 1, 0, 0], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

boston_merged = dataframe_filtered

# merge boston_grouped with boston_data to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

boston_merged = boston_merged.dropna()


boston_merged # check the last columns!

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,...,country,formattedAddress,neighborhood,id,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
43,Dunkin',Donut Shop,16 Tremont St,City Hall Plaza,42.358929,-71.059637,"[{'label': 'display', 'lat': 42.3589286, 'lng'...",184,02108,US,...,United States,"[16 Tremont St (City Hall Plaza), Boston, MA 0...",Downtown Boston,4b9a5171f964a520f3ab35e3,1,Coffee Shop,Historic Site,American Restaurant,Sandwich Place,Pub
89,Spicy Salaa,Food Truck,City Hall Plaza,Fisher Park,42.359572,-71.057533,"[{'label': 'display', 'lat': 42.35957166573581...",98,02203,US,...,United States,"[City Hall Plaza (Fisher Park), Boston, MA 022...",Downtown Boston,5925b4dd6fd62668ebf46ba9,1,Italian Restaurant,Historic Site,Coffee Shop,Bakery,Pub


In [26]:
# create map
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['lat'], boston_merged['lng'], boston_merged['name'], boston_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.5               |   py36h830a2c2_0        10.1 MB  conda-forge
    toolz-0.10.0               |             py_0          46 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        10.8 MB

The following NEW p

In [27]:
boston_merged.loc[boston_merged['ClusterLabels'] ==0, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]

,categories,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


# 4. Results & Discussion

According to the result of our analysis, downtown Boston (in cluster 1) is the best neighborhood to open an ice cream shop in Boston. It has the greatest number of total venues and is expected to attract many people to visit. This neighborhood with Park, city plaza hall and no ice cream shop. However, the final decision is dependent on the budget of the client to rent the place. According to the budget limitations of the client, we can find other neighborhoods and select the one with the greatest number of venues and a smaller number of ice cream shops around. This clustering can also help our client compare different neighborhoods to make a better decision

# 5. Conclusion

Used the information about the neighborhoods of Boston to help my client to select the best location to open an ice cream shop in Boston. Considered number of competitors, demand for ice cream, and budget limitations to find the best location options. This project has some limitations. First, we have limited the location options to the neighborhoods of Boston, however considering more specific locations such as apartmental Areas can improve the accuracy of our decision. Second, we used the median of one bedroom rent price in each neighborhood as an estimate of the rent price. Third, we limited our investigation for the venues within 500 meter of the neighborhoods and changing it can affect the clusters. Over all Good learning and understood better on a activities needs to be performed by a Data scientist.

## Limitations:

The current limitation for number of clustering. the results may vary based on the cluster selection and distance range mentioned in neighborhood.